# Exercises

Within your codeup-data-science directory, create a new directory named clustering-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save your clustering work in your clustering-exercises repo. Then add, commit, and push your changes.

For example, if the exercise directs you to create a file named myfile.py, you should have clustering/myfile.py in your repository.

If a file extension is specified, you should create that specific file. If there is not file extension specified, you may either create a python script or a jupyter notebook for the exercise.

In [1]:
# Imports
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import summarize
import wrangle

## Acquire & Summarize

1) Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
        
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
        
- Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
        
- Only include properties that include a latitude and longitude value.

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)
   
3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [2]:
df = acquire.get_zillow_data()

df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,1072280,10711877,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,1340933,10711888,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,1878109,10711910,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,2190858,10711923,nan,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [3]:
df = df.drop(columns = ['id'])

In [4]:
from summarize import df_summary
info, describe, nulls, value_count = df_summary (df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72236 entries, 0 to 72235
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72236 non-null  int64  
 1   airconditioningtypeid         23078 non-null  float64
 2   architecturalstyletypeid      206 non-null    float64
 3   basementsqft                  50 non-null     float64
 4   bathroomcnt                   72236 non-null  float64
 5   bedroomcnt                    72236 non-null  float64
 6   buildingclasstypeid           0 non-null      object 
 7   buildingqualitytypeid         45079 non-null  float64
 8   calculatedbathnbr             71983 non-null  float64
 9   decktypeid                    612 non-null    float64
 10  finishedfloor1squarefeet      6030 non-null   float64
 11  calculatedfinishedsquarefeet  72051 non-null  float64
 12  finishedsquarefeet12          71871 non-null  float64
 13  f

## Prepare

1) Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2) Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

### The input:
            
- A dataframe
            
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
            
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
        
### The output:
            
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
       
### hint:
            
- Look up the dropna documentation.
            
- You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
            
- Make use of inplace, i.e. inplace=True/False.

3) Decide how to handle the remaining missing values:
         - Fill with constant value.
         - Impute with mean, median, mode.
         - Drop row/column


### wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.
Mall Customers

In [6]:
from wrangle import handle_missing_values

In [7]:
df = handle_missing_values(df, prop_required_column = .5, prop_required_row = .75)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72059 entries, 0 to 72235
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72059 non-null  int64  
 1   bathroomcnt                   72059 non-null  float64
 2   bedroomcnt                    72059 non-null  float64
 3   buildingqualitytypeid         45078 non-null  float64
 4   calculatedbathnbr             71979 non-null  float64
 5   calculatedfinishedsquarefeet  72046 non-null  float64
 6   finishedsquarefeet12          71867 non-null  float64
 7   fips                          72059 non-null  float64
 8   fullbathcnt                   71979 non-null  float64
 9   heatingorsystemtypeid         46695 non-null  float64
 10  latitude                      72059 non-null  float64
 11  longitude                     72059 non-null  float64
 12  lotsizesquarefeet             63970 non-null  float64
 13  p

# notebook

- Acquire data from mall_customers.customers in mysql database.
- Summarize data (include distributions and descriptive statistics).
- Detect outliers using IQR.
- Split data (train, validate, and test split).
- Encode categorical columns using a one hot encoder.
- Handles missing values.
- Scaling


In [8]:
df = acquire.get_zillow_data()

df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1087254,10711855,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,1072280,10711877,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,1340933,10711888,1.00,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,1878109,10711910,nan,nan,nan,2.00,3.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,2190858,10711923,nan,nan,nan,2.00,4.00,None,8.00,2.00,...,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [9]:
df = df.drop(columns = ['id'])

In [10]:
from summarize import df_summary
info, describe, nulls, value_count = df_summary (df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72236 entries, 0 to 72235
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72236 non-null  int64  
 1   airconditioningtypeid         23078 non-null  float64
 2   architecturalstyletypeid      206 non-null    float64
 3   basementsqft                  50 non-null     float64
 4   bathroomcnt                   72236 non-null  float64
 5   bedroomcnt                    72236 non-null  float64
 6   buildingclasstypeid           0 non-null      object 
 7   buildingqualitytypeid         45079 non-null  float64
 8   calculatedbathnbr             71983 non-null  float64
 9   decktypeid                    612 non-null    float64
 10  finishedfloor1squarefeet      6030 non-null   float64
 11  calculatedfinishedsquarefeet  72051 non-null  float64
 12  finishedsquarefeet12          71871 non-null  float64
 13  f

In [12]:
from wrangle import handle_missing_values
df = handle_missing_values(df, prop_required_column = .5, prop_required_row = .75)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72059 entries, 0 to 72235
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      72059 non-null  int64  
 1   bathroomcnt                   72059 non-null  float64
 2   bedroomcnt                    72059 non-null  float64
 3   buildingqualitytypeid         45078 non-null  float64
 4   calculatedbathnbr             71979 non-null  float64
 5   calculatedfinishedsquarefeet  72046 non-null  float64
 6   finishedsquarefeet12          71867 non-null  float64
 7   fips                          72059 non-null  float64
 8   fullbathcnt                   71979 non-null  float64
 9   heatingorsystemtypeid         46695 non-null  float64
 10  latitude                      72059 non-null  float64
 11  longitude                     72059 non-null  float64
 12  lotsizesquarefeet             63970 non-null  float64
 13  p

In [23]:
df.isnull().sum()

parcelid                           24
bathroomcnt                      2367
bedroomcnt                        148
buildingqualitytypeid           28112
calculatedbathnbr                2381
calculatedfinishedsquarefeet     3816
finishedsquarefeet12             3989
fips                             6101
fullbathcnt                      1665
heatingorsystemtypeid           26338
latitude                           42
longitude                        1872
lotsizesquarefeet               18634
propertycountylandusecode       72059
propertylandusetypeid               0
propertyzoningdesc              72059
rawcensustractandblock           6101
regionidcity                     3268
regionidcounty                      0
regionidzip                        48
roomcnt                         72059
unitcnt                         72059
yearbuilt                         178
structuretaxvaluedollarcnt       5145
taxvaluedollarcnt                4822
assessmentyear                  72059
landtaxvalue

In [13]:
def IQR_limit(df):    
    q1 = df.quantile(.25)
    q3 = df.quantile(.75)

    iqr = q3 - q1

    multiplier = 1.5
    upper_bound = q3 + (multiplier * iqr)
    lower_bound = q1 - (multiplier * iqr)

    # Let's filter out the low outliers
    df = df[df > lower_bound]
    # lets say give us everyhting less than the upper bound
    df = df[df < upper_bound]

    return df


In [14]:
df = IQR_limit(df)
df.shape

(72059, 33)

In [17]:
from wrangle import data_split
X_train, y_train, X_validate, y_validate, X_test, y_test = data_split(df, stratify_by='taxvaluedollarcnt')
X_train.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,...,yearbuilt,structuretaxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
37099,"12,643,333.00",2.00,2.00,8.00,2.00,"1,087.00","1,087.00","6,037.00",2.00,2.00,...,"1,987.00","25,532.00",nan,"63,599.00","1,074.99","60,372,963,002,006.00",-0.01,NaN,NaN,NaN
68622,"17,163,867.00",2.00,4.00,nan,2.00,"1,717.00","1,717.00",nan,2.00,nan,...,"1,978.00","126,875.00",nan,"42,276.00","1,974.10",nan,0.00,NaN,NaN,NaN
45065,"13,071,085.00",3.00,4.00,8.00,3.00,"2,291.00","2,291.00","6,037.00",3.00,2.00,...,"1,980.00","211,533.00",nan,"186,083.00","5,284.09","60,374,033,183,008.00",nan,NaN,NaN,NaN
29995,"12,221,646.00",1.00,2.00,5.00,1.00,"1,080.00","1,080.00","6,037.00",1.00,nan,...,"1,928.00","16,436.00",nan,"21,277.00",635.21,"60,372,405,003,004.00",0.02,NaN,NaN,NaN
31612,"12,335,097.00",1.00,2.00,6.00,1.00,"1,504.00","1,504.00","6,037.00",1.00,7.00,...,"1,950.00","133,902.00",nan,"177,943.00","3,962.85","60,375,512,011,004.00",0.02,NaN,NaN,NaN


In [18]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40352 entries, 37099 to 27733
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      40335 non-null  float64
 1   bathroomcnt                   38989 non-null  float64
 2   bedroomcnt                    40264 non-null  float64
 3   buildingqualitytypeid         24619 non-null  float64
 4   calculatedbathnbr             38982 non-null  float64
 5   calculatedfinishedsquarefeet  38184 non-null  float64
 6   finishedsquarefeet12          38090 non-null  float64
 7   fips                          36912 non-null  float64
 8   fullbathcnt                   39402 non-null  float64
 9   heatingorsystemtypeid         25620 non-null  float64
 10  latitude                      40326 non-null  float64
 11  longitude                     39324 non-null  float64
 12  lotsizesquarefeet             29840 non-null  float64
 1

In [19]:
import sklearn.preprocessing

# wrangle_mall.py

- Acquire data from mall_customers.customers in mysql database.
- Split the data into train, validate, and split
- One-hot-encoding
- Missing values
- Scaling